# Set up local SparkSession

In [12]:
from datetime import date

from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F

from delta.tables import DeltaTable

In [13]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)
    .getOrCreate()
)

# Read flights data from CSV

In [14]:
df = (
    spark
    .read
    .option("header", True)
    .option("inferSchema", True)
    .csv("flights.csv")
)
df

year,month,day,departure_time,arrival_time,carrier,flight_number,aircraft_registration,flight_time,airport_origin,airport_destination,distance,is_cancelled,is_diverted
2008,1,3,1343,1451,WN,588,N240WN,68,HOU,LIT,393,0,0
2008,1,3,1125,1247,WN,1343,N523SW,82,HOU,MAF,441,0,0
2008,1,3,2009,2136,WN,3841,N280WN,87,HOU,MAF,441,0,0
2008,1,3,903,1203,WN,3,N308SA,120,HOU,MCO,848,0,0
2008,1,3,1423,1726,WN,25,N462WN,123,HOU,MCO,848,0,0
2008,1,3,2024,2325,WN,51,N483WN,121,HOU,MCO,848,0,0
2008,1,3,1753,2053,WN,940,N493WN,120,HOU,MCO,848,0,0
2008,1,3,622,935,WN,2621,N266WN,133,HOU,MCO,848,0,0
2008,1,3,1944,2210,WN,389,N266WN,146,HOU,MDW,937,0,0
2008,1,3,1453,1716,WN,519,N514SW,143,HOU,MDW,937,0,0


# Perform some simple aggregations and transformations

In [15]:
df = (
    df
    .withColumn("date", F.to_date(F.concat("year", F.lit("-"), "month", F.lit("-"), "day")))
    .where("is_cancelled = 0 and is_diverted = 0")
    .groupBy("date", "airport_origin", "airport_destination")
    .agg({"*": "count", "flight_time": "avg"})
    .withColumnRenamed("count(1)", "count")
    .withColumnRenamed("avg(flight_time)", "avg_flight_time")
    .withColumn("avg_flight_time", F.col("avg_flight_time").cast("int"))
)
df.orderBy("count", ascending=False)

date,airport_origin,airport_destination,count,avg_flight_time
2008-01-03,LAX,OAK,20,80
2008-01-03,LAS,PHX,18,64
2008-01-03,LAX,LAS,14,63
2008-01-03,LAS,LAX,14,67
2008-01-03,LAS,SAN,14,63
2008-01-03,LAS,BUR,13,62
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,LAS,RNO,13,74
2008-01-03,LAS,OAK,13,95


# Write to partitioned Delta table

In [16]:
(
    df
    .coalesce(5)
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy(["date"])
    .format("delta")
    .saveAsTable("flights")
)

# Perform updates, deletes and merges on the table

In [17]:
flights_table = DeltaTable.forName(spark, "flights")

In [18]:
flights_table.update(
    condition="avg_flight_time = 80",
    set={"avg_flight_time": "null"}
)

flights_table.delete("airport_origin = 'LAS' or airport_destination = 'LAS'")

flights_table.toDF().orderBy("count", ascending=False)

date,airport_origin,airport_destination,count,avg_flight_time
2008-01-03,LAX,OAK,20,null
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,MCI,MDW,12,81
2008-01-03,MDW,MCI,12,79
2008-01-03,HOU,MSY,11,58
2008-01-03,MHT,BWI,11,79
2008-01-03,LAX,SMF,10,71
2008-01-03,MDW,STL,10,59
2008-01-03,HRL,HOU,9,54


In [19]:
incoming_df = spark.createDataFrame([
    {"date": date(2008, 1, 3), "airport_origin": "LAX", "airport_destination": "OAK", "count": 21, "avg_flight_time": 83},
    {"date": date(2008, 1, 4), "airport_origin": "JFK", "airport_destination": "EZE", "count": 22, "avg_flight_time": 660},
])

(
    flights_table
    .alias("existing")
    .merge(
        source=incoming_df.alias("incoming"),
        condition="existing.date = incoming.date "
                  "AND existing.airport_origin = incoming.airport_origin "
                  "AND existing.airport_destination = incoming.airport_destination"
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)

flights_table.toDF().orderBy("count", ascending=False)

date,airport_origin,airport_destination,count,avg_flight_time
2008-01-04,JFK,EZE,22,660
2008-01-03,LAX,OAK,21,83
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,MCI,MDW,12,81
2008-01-03,MDW,MCI,12,79
2008-01-03,MHT,BWI,11,79
2008-01-03,HOU,MSY,11,58
2008-01-03,MDW,STL,10,59
2008-01-03,LAX,SMF,10,71


# Time travel

In [20]:
flights_table.history()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2022-05-20 14:39:51,null,null,MERGE,{predicate -> (((...,null,null,null,2,Serializable,false,{numTargetRowsCop...,null,Apache-Spark/3.2....
2,2022-05-20 14:39:47,null,null,DELETE,"{predicate -> [""(...",null,null,null,1,Serializable,false,{numRemovedFiles ...,null,Apache-Spark/3.2....
1,2022-05-20 14:39:45,null,null,UPDATE,{predicate -> (av...,null,null,null,0,Serializable,false,{numRemovedFiles ...,null,Apache-Spark/3.2....
0,2022-05-20 14:39:41,null,null,CREATE OR REPLACE...,{isManaged -> tru...,null,null,null,null,Serializable,false,"{numFiles -> 1, n...",null,Apache-Spark/3.2....


In [21]:
df = spark.read.option("versionAsOf", 0).table("flights")
df.orderBy("count", ascending=False)

date,airport_origin,airport_destination,count,avg_flight_time
2008-01-03,LAX,OAK,20,80
2008-01-03,LAS,PHX,18,64
2008-01-03,LAX,LAS,14,63
2008-01-03,LAS,LAX,14,67
2008-01-03,LAS,SAN,14,63
2008-01-03,LAS,BUR,13,62
2008-01-03,LAX,PHX,13,66
2008-01-03,LAX,SJC,13,64
2008-01-03,LAS,RNO,13,74
2008-01-03,LAS,OAK,13,95


# Vacuum 

In [22]:
# Keep only last 30 days of data
flights_table.vacuum(retentionHours=720)

Deleted 0 files and directories in a total of 3 directories.


""


# Schema evolution

In [23]:
# TODO